<a href="https://colab.research.google.com/github/Edudeiko/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module1-statistics-probability-and-inference/Evgenii_Dudeiko_DSPT3_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
cv_data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

In [0]:
import pandas as pd
cv_data = pd.read_csv(cv_data_url)

In [204]:
cv_data.head()

,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [205]:
cv_data.count()

republican    434
n             434
y             434
n.1           434
y.1           434
y.2           434
y.3           434
n.2           434
n.3           434
n.4           434
y.4           434
?             434
y.5           434
y.6           434
y.7           434
n.5           434
y.8           434
dtype: int64

In [206]:
cv_data = pd.read_csv(cv_data_url, header=None)
cv_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [207]:
cv_data.shape

(435, 17)

In [208]:
column_headers = ['Class Name', 'handicapped-infants', 
                  'water-project-cost-sharing', 
                  'adoption-of-the-budget-resolution', 'physician-fee-freeze', 
                  'el-salvador-aid', 'religious-groups-in-schools', 
                  'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 
                  'mx-missile', 'immigration', 'synfuels-corporation-cutback',
                  'education-spending', 'superfund-right-to-sue', 'crime',
                  'duty-free-exports', 'export-administration-act-south-africa']

df = pd.read_csv(cv_data_url, names=column_headers)
print(df.shape)
df.head()

(435, 17)


,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [209]:
df['Class Name'].unique()

array(['republican', 'democrat'], dtype=object)

In [210]:
df.set_index('Class Name')

,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
Class Name,,,,,,,,,,,,,,,,
republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y


In [211]:
df.shape

(435, 17)

In [0]:
df['handicapped-infants'] = df['handicapped-infants'].str.contains('y')

In [0]:
df['water-project-cost-sharing'] = df['water-project-cost-sharing'].str.contains('y')

In [0]:
df['adoption-of-the-budget-resolution'] = df['adoption-of-the-budget-resolution'].str.contains('y')

In [0]:
df['physician-fee-freeze'] = df['physician-fee-freeze'].str.contains('y')

In [0]:
df['el-salvador-aid'] = df['el-salvador-aid'].str.contains('y')
df['religious-groups-in-schools'] = df['religious-groups-in-schools'].str.contains('y')
df['anti-satellite-test-ban'] = df['anti-satellite-test-ban'].str.contains('y')
df['aid-to-nicaraguan-contras'] = df['aid-to-nicaraguan-contras'].str.contains('y')
df['mx-missile'] = df['mx-missile'].str.contains('y')
df['immigration'] = df['immigration'].str.contains('y')
df['synfuels-corporation-cutback'] = df['synfuels-corporation-cutback'].str.contains('y')
df['education-spending'] = df['education-spending'].str.contains('y')
df['superfund-right-to-sue'] = df['superfund-right-to-sue'].str.contains('y')
df['crime'] = df['crime'].str.contains('y')
df['duty-free-exports'] = df['duty-free-exports'].str.contains('y')
df['export-administration-act-south-africa'] = df['export-administration-act-south-africa'].str.contains('y')


In [217]:
df.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,False,True,False,True,True,True,False,False,False,True,False,True,True,True,False,True
1,republican,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False
2,democrat,False,True,True,False,True,True,False,False,False,False,True,False,True,True,False,False
3,democrat,False,True,True,False,False,True,False,False,False,False,True,False,True,False,False,True
4,democrat,True,True,True,False,True,True,False,False,False,False,True,False,True,True,True,True


In [218]:
df['handicapped-infants'].value_counts()

False    248
True     187
Name: handicapped-infants, dtype: int64

In [219]:
df['Class Name'].value_counts()


democrat      267
republican    168
Name: Class Name, dtype: int64

In [0]:
# Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

In [220]:
df['Class Name'].unique()

array(['republican', 'democrat'], dtype=object)

In [221]:
df_republican = df[df['Class Name'] == 'republican']
df_republican.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,False,True,False,True,True,True,False,False,False,True,False,True,True,True,False,True
1,republican,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False
7,republican,False,True,False,True,True,True,False,False,False,False,False,False,True,True,False,True
8,republican,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,True
10,republican,False,True,False,True,True,False,False,False,False,False,False,False,True,True,False,False


In [222]:
df_republican.shape

(168, 17)

In [223]:
df_democrat = df[df['Class Name'] == 'democrat']
df_democrat.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
2,democrat,False,True,True,False,True,True,False,False,False,False,True,False,True,True,False,False
3,democrat,False,True,True,False,False,True,False,False,False,False,True,False,True,False,False,True
4,democrat,True,True,True,False,True,True,False,False,False,False,True,False,True,True,True,True
5,democrat,False,True,True,False,True,True,False,False,False,False,False,False,True,True,True,True
6,democrat,False,True,False,True,True,True,False,False,False,False,False,False,False,True,True,True


In [224]:
df_democrat.shape

(267, 17)

In [0]:
# democrats support more than republicans with p < 0.01

In [171]:
df_democrat_group = df_democrat.groupby('adoption-of-the-budget-resolution')['handicapped-infants'].mean()
df_democrat_group

adoption-of-the-budget-resolution
False    0.388889
True     0.614719
Name: handicapped-infants, dtype: float64

In [173]:
df_democrat[df_democrat['adoption-of-the-budget-resolution'] == True]['handicapped-infants'].mean()

0.6147186147186147

In [176]:
df_republican_group = df_republican.groupby('adoption-of-the-budget-resolution')['handicapped-infants'].mean()
df_republican_group

adoption-of-the-budget-resolution
False    0.150685
True     0.409091
Name: handicapped-infants, dtype: float64

In [177]:
df_republican[df_republican['adoption-of-the-budget-resolution'] == True]['handicapped-infants'].mean()

0.4090909090909091

In [178]:
ttest_ind(df_republican_group, df_democrat_group)

Ttest_indResult(statistic=-1.293290059264266, pvalue=0.32514701510245503)

In [152]:
df_democrat[df_democrat['crime'] == True].mean()

handicapped-infants                       0.444444
water-project-cost-sharing                0.477778
adoption-of-the-budget-resolution         0.711111
physician-fee-freeze                      0.144444
el-salvador-aid                           0.500000
religious-groups-in-schools               0.822222
anti-satellite-test-ban                   0.500000
aid-to-nicaraguan-contras                 0.566667
mx-missile                                0.500000
immigration                               0.544444
synfuels-corporation-cutback              0.600000
education-spending                        0.322222
superfund-right-to-sue                    0.522222
crime                                     1.000000
duty-free-exports                         0.366667
export-administration-act-south-africa    0.600000
dtype: float64

In [153]:
df_republican[df_republican['crime'] == True].mean()

handicapped-infants                       0.189873
water-project-cost-sharing                0.424051
adoption-of-the-budget-resolution         0.132911
physician-fee-freeze                      0.981013
el-salvador-aid                           0.943038
religious-groups-in-schools               0.886076
anti-satellite-test-ban                   0.240506
aid-to-nicaraguan-contras                 0.145570
mx-missile                                0.113924
immigration                               0.556962
synfuels-corporation-cutback              0.120253
education-spending                        0.816456
superfund-right-to-sue                    0.822785
crime                                     1.000000
duty-free-exports                         0.082278
export-administration-act-south-africa    0.594937
dtype: float64

In [133]:
df_democrat_group = df_democrat.groupby('handicapped-infants')['education-spending'].mean()
df_democrat_group

handicapped-infants
False    0.198198
True     0.089744
Name: education-spending, dtype: float64

In [134]:
df_democrat[df_democrat['handicapped-infants'] == True]['education-spending'].mean()

0.08974358974358974

In [0]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [135]:
ttest_ind(df_republican_group, df_democrat_group)

Ttest_indResult(statistic=3.899110181106702, pvalue=0.059924676000885004)

In [0]:
# Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01